# Talos Example

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import numpy as np
import matplotlib.pyplot as plt

from collections import OrderedDict

import tf_robot_learning as rl
import tf_robot_learning.distributions as ds
import os, time
from tf_robot_learning import kinematic as tk

from IPython.core import display

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Part 1: Define and Train GAN

In [ ]:
sess = tf1.InteractiveSession()
tf1.disable_eager_execution()

### Define robot

In [ ]:
urdf = tk.urdf_from_file(rl.datapath + '/urdf/talos_reduced.urdf');
display.clear_output()

#list of end-effector
tips = OrderedDict({
    'r_gripper'	: 'gripper_right_base_link',
    'l_gripper'	: 'gripper_left_base_link',
    'r_foot' 	: 'right_sole_link',
    'l_foot' 	: 'left_sole_link',
})

#define the robot as a kinematic chain, loaded from urdf
chain_names = ['r_gripper', 'r_foot', 'l_foot'] 
chain = tk.ChainDict({
    name: tk.kdl_chain_from_urdf_model(urdf, 'base_link', tip=tip)
    for name, tip in tips.items()
})

#define the default position and orientation of the end-effector
ee = OrderedDict({
        'l_gripper': [0.  , 0.29, 0.8 , 1.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ,1.  ],
        'r_gripper': [0.  , -0.29, 0.8 , 1.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ,1.  ],
        'l_foot': [-0.02,  0.09, -0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  , 0.  ,  0.  ,  1.  ],
        'r_foot': [-0.02,  -0.09, -0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  , 0.  ,  0.  ,  1.  ],
})

### Load data 

In [ ]:
def param_to_joint_pos(x):
    """
    convert the config parameter x to full robot configurations: joint angles, base position and orientation
    """
    return x[..., :chain.nb_joint],\
        tf.concat([x[..., chain.nb_joint:chain.nb_joint+3]], -1),\
        tk.rotation.rpy(tf.zeros_like(x[..., -3:]))

In [ ]:
def q_augmented(q):
    """ Define augmented data transformations: (joint_angles, right foot pose, and left foot pose)"""
    _q, _p, _m = param_to_joint_pos(q)
    return tf.concat([
            q,
            chain.xs(_q, floating_base=(_p, _m), name='r_foot')[:, -1],
            chain.xs(_q, floating_base=(_p, _m), name='l_foot')[:, -1]
        ], -1)

In [ ]:
def q_target(q):
    """ Extract the target"""
    _q, _p, _m = param_to_joint_pos(q)
    return tf.concat( # orientation of each foot
            [chain.xs(_q, floating_base=(_p, _m), name=name)[:, -1, :3] for name in chain_names]
            , -1)

In [ ]:
def q_foot_ori(q):
    """ Extract the foot orientation """
    _q, _p, _m = param_to_joint_pos(q)
    return tf.concat( # position of each end-effector
            [chain.xs(_q, floating_base=(_p, _m), name=name)[:, -1, 3:] for name in chain_names[1:]]
            , -1)

In [ ]:
# get mini batch
def get_batch(_batch_size=30, cut=None, augmented=True):
    if cut is not None: idx = np.random.randint(0, cut, _batch_size)
    else: idx = np.random.randint(0, data_augmented.shape[0]-1, _batch_size)
    if augmented: return data_augmented[idx]
    else: return data[idx]
    
def get_target_batch(_batch_size=30, cut=None):
    if cut is not None: idx = np.random.randint( 0, cut, _batch_size)
    else: idx = np.random.randint(0, data_augmented.shape[0]-1, _batch_size)
    return data_target[idx]


In [ ]:
q_eval = tf1.placeholder(tf.float32, (None, chain.nb_joint + 3))

q_augmented_eval = q_augmented(q_eval)
q_target_eval = q_target(q_eval)
q_foot_ori_eval = q_foot_ori(q_eval)

In [ ]:
#load the dataset
data = np.load('data/data_two_feet_manual.npy')
# compute data through these transformations
data_augmented = q_augmented_eval.eval({q_eval: data})
data_target = q_target_eval.eval({q_eval: data})

## Generator 

In [ ]:
# Network Parameters
joint_dim = chain.nb_joint + 3 # the configuration consists of nb_joint= 28 joint angles and 3 base position
latent_dim = 30 # dimension of noise
N_net = 5
target_dim = 9  # size of the target
augmented_dim = 2 * 12 + joint_dim # size of augmented data (joint angles + poses of both foot)

n_input = latent_dim + target_dim #dimension of the input to the generator
batch_size = tf1.placeholder(tf.int32, ())

#### Define the generator

In [ ]:
gen_nn = rl.nn.MLP(
    n_input=n_input, n_output=joint_dim, n_hidden=[200, 200],
    act_fct=tf.nn.relu, batch_size_svi=N_net
)

#### Define the generator input

In [ ]:
# noise to feed generator
eps_var = tf1.placeholder(tf.float32, ())
eps = tf.random.normal([tf.cast(batch_size/N_net, tf.int32), latent_dim], 
    dtype=tf.float32, mean=0., stddev=eps_var, name='epsilon')

#the input to generator = noise + target
batch_target = tf1.placeholder(tf.float32, (None, target_dim))
eps_conc = tf.concat([eps, batch_target], axis=-1)

#### Define output transformations

In [ ]:
samples_q = tf.reshape(gen_nn.pred(eps_conc) + tf.constant(chain.mean_pose + [0,0,1.08])[None], (-1, joint_dim))

samples_qq, samples_p, samples_m = param_to_joint_pos(samples_q)
samples_links, _, samples_com = chain.xs(samples_qq, floating_base=(samples_p, samples_m), get_links=True)

samples_augmented = q_augmented(samples_q)
samples_target = q_target(samples_q)
samples_foot_ori = q_foot_ori(samples_q)

## Discriminator

#### Define the discriminator

In [ ]:
discr_nn = rl.nn.MLP(
    n_input=augmented_dim, n_output=1, n_hidden=[40, 40],
    act_fct=tf.nn.relu
)

#batch data
batch_x = tf1.placeholder(tf.float32, (None, augmented_dim))

## Define loss functions

#### Main loss functions (discriminator vs generator)

In [ ]:
d_fake = discr_nn.pred(samples_augmented)[:, 0]
d_true = discr_nn.pred(batch_x)[:, 0]

#discriminator loss function
loss_d = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_true), logits=d_true) + \
        tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(d_fake), logits=d_fake)
loss_d = tf.reduce_sum(loss_d)

#generator (basic) loss function
loss = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=tf.ones_like(d_fake), logits=d_fake)
loss = tf.reduce_sum(loss)

#### Target loss 

In [ ]:
p_target_std = tf1.placeholder(tf.float32, ()) 

p_target = ds.MultivariateNormalFullCovariance(
    tf.reshape(tf.ones((N_net, 1,1))*batch_target,(-1, target_dim)), p_target_std**2 * tf.eye(target_dim)
)

loss_target = tf.reduce_sum(-p_target.log_prob(samples_target))
lmbda_target = tf1.placeholder(tf.float32, ())

#### Foot orientation loss 

In [ ]:
foot_ori_dim = 18
batch_foot_ori = tf1.placeholder(tf.float32, (None, foot_ori_dim))

# define a Gaussian distribution that should be tracked by the system
p_foot_ori_std = tf1.placeholder(tf.float32, ()) 

p_foot_ori = ds.MultivariateNormalFullCovariance(
    tf.reshape(tf.ones((N_net, 1,1))*batch_foot_ori,(-1, foot_ori_dim)), p_foot_ori_std**2 * tf.eye(foot_ori_dim)
)

loss_foot_ori = tf.reduce_sum(-p_foot_ori.log_prob(samples_foot_ori))
lmbda_foot_ori = tf1.placeholder(tf.float32, ())

#### Constraints cost: COM and joint limit

In [ ]:
#joint limit
base_pos_limits = np.array([[-0.1, -0.1, 0.6],[0.1, 0.1, 1.4] ]).T
config_limits = np.concatenate([chain.joint_limits, base_pos_limits], axis=0)
joint_limits = tf.constant(config_limits, dtype=tf.float32)
joint_limits_std = 0.05
joint_limits_temp = 1.

joint_limits_exp = ds.SoftUniformNormalCdf(
    low=joint_limits[:, 0],
    high=joint_limits[:, 1],
    std=joint_limits_std,
    temp=joint_limits_temp,
    reduce_axis=-1
)

joint_limit_constraints = tf.reduce_mean(-joint_limits_exp.log_prob(samples_q[:,:chain.nb_joint+3]))


#COM
com_limits = 0.1
com_limits_std = 0.01

com_limits_exp = ds.SoftUniformNormalCdf(
    low=-com_limits,
    high=com_limits,
    std=com_limits_std,
    temp=1.,
    reduce_axis=-1
)

com_xy = samples_com[:, :2]
center_feet = tf.reduce_mean([
        samples_links['r_foot'][:, -1, :2], 
        samples_links['l_foot'][:, -1, :2]
        ], axis=0)

cost_constraints = tf.reduce_mean(-com_limits_exp.log_prob(com_xy - center_feet))
lmbda_constraints = tf1.placeholder(tf.float32, ())

### Overall loss

In [ ]:
loss_gen = loss +  lmbda_target * loss_target + lmbda_foot_ori*loss_foot_ori + lmbda_constraints * (cost_constraints+joint_limit_constraints) 

### Optimizer 

In [ ]:
rate = tf1.placeholder(tf.float32, ())
opt = tf1.train.AdamOptimizer

optimizer = opt(learning_rate=rate)
optimizer_d = opt(learning_rate=rate)

train = optimizer.minimize(loss_gen, var_list=gen_nn.vec_weights)
train_d = optimizer_d.minimize(loss_d, var_list=discr_nn.vec_weights)

# Initialize the variables (i.e. assign their default value)
init = tf1.global_variables_initializer()

## Training model 

In [ ]:
sess.run(init)

In [ ]:
def get_target_new_batch(_batch_size):
    return np.random.multivariate_normal(_targets_m, np.diag(_targets_v ** 2), (_batch_size, ))

In [ ]:
_batch_size = 100
_targets_m = np.concatenate([ee[name][:3] for name in chain_names], 0) # mean target
_batch_foot_ori = np.tile(np.eye(3).flatten(), (int(_batch_size/N_net),2)).reshape(int(_batch_size/N_net),-1)

_targets_m = np.concatenate([ee[name][:3] for name in chain_names], 0) # mean target
_targets_v = np.array([0.5] * 3 + [0.01] * 2 + [0.] +  [0.01] * 2 + [0.]  )  # variance

In [ ]:
alpha = .7
for i in range(2000):
    try:
        for j in range(5):
            # train discriminative_network
            _x = get_batch(cut=8000, _batch_size=_batch_size)
            
            feed_dict = {
                lmbda_target: .05,
                lmbda_foot_ori: .05,
                lmbda_constraints: 5.,
                eps_var: 1.,
                p_target_std: 0.01,
                p_foot_ori_std: 0.01,
                batch_x: _x,
                batch_size: _batch_size,
                rate : 0.002 * alpha,
            }
            
            # from data
            feed_dict[batch_target] = get_target_new_batch(_batch_size=int(_batch_size/N_net))

            _ = sess.run([train_d], feed_dict=feed_dict)
        
        feed_dict[rate] = .001 * alpha
        feed_dict[batch_foot_ori] = _batch_foot_ori
        
        # train generative_network
        _,  _loss, _loss_d,  _loss_target, _loss_constraints = sess.run(
            [train, loss, loss_d,  loss_target, cost_constraints], feed_dict=feed_dict)
        
        if not i % 10:
            display.clear_output(wait=True)
            print('Step %i\t, loss gen %f, loss disc %f,  loss target %f, loss constraints %f' % (i, _loss, _loss_d ,_loss_target, _loss_constraints))
    except KeyboardInterrupt:
        break

#### To save model

#### To load model

In [ ]:
saver = tf1.train.Saver()
saver.restore(sess, "data/talos_foot_fixed_with_data_ensemble2.ckpt")

### Visualize the generated samples

In [ ]:
_batch_size = 20
feed_dict={batch_size: _batch_size, eps_var:1.}
feed_dict[batch_target] = get_target_new_batch(_batch_size=int(_batch_size/N_net))

_links, _com , _b_targets = sess.run(
[samples_links, samples_com, batch_target], feed_dict
)

fig, ax = plt.subplots(ncols=2)

for i in range(2):
    dim = [i, 2]
    chain.plot(
            _links, feed_dict={}, ax=ax[i],
            dim=dim, alpha=0.2, color='k'
        )    
    ax[i].plot(_com[0, dim[0]], _com[0, dim[1]], 'bx')
    
    for j, name in enumerate(chain_names):
        ax[i].plot(_b_targets[:, j*3 + dim[0]], _b_targets[:, j*3 + dim[1]], 'rx', color='orangered')

In [ ]:
from_dataset = 0
n = 10

_targets_m = np.concatenate([ee[name][:3] for name in chain_names], 0) # mean target
_targets_v = np.array([0.6] * 3 + [0.001]*3 + [0.001] * 3  )  # variance

if from_dataset:
    _targets = get_target_batch(cut=10000, _batch_size=1) * np.ones((int(n/N_net), 1))
else:
    _targets = np.random.multivariate_normal(_targets_m, np.diag(_targets_v ** 2), (1, )) * np.ones((int(n/N_net), 1))

feed_dict={batch_size: n, batch_target: _targets, eps_var:1.}


_links, _com , _b_targets, _samples_q = sess.run(
    [samples_links, samples_com, batch_target, samples_q], 
    feed_dict)

fig, ax = plt.subplots(ncols=2, figsize=(12,8))

for i in range(2):
    dim = [i, 2]
    chain.plot(
            _links, feed_dict={}, ax=ax[i],
            dim=dim, alpha=0.2, color='k'
        )    
    
    for j, name in enumerate(chain_names):
        ax[i].plot(_b_targets[:, j*3 + dim[0]], _b_targets[:, j*3 + dim[1]], 'rx', color='orangered')
    ax[i].plot(_com[0, dim[0]], _com[0, dim[1]], 'bx')
    #ax[i].set_xlim([-.7, .7])
    #x[i].set_ylim([-0.1, 1.2])
plt.savefig('data/talos_config.png')
plt.show()  


## Part 2: Use GAN for Projection, IK and Inverse Kinematics

### Setup Pybullet

In [ ]:
import pybullet as p
import pybullet_data
from utils import *
from scipy.optimize import fmin_bfgs
from costs import *
from robot import *
from scipy.optimize import fmin_bfgs
from functools import partial

In [ ]:
physics_client_id = p.connect(p.GUI)

p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

In [ ]:
p.resetSimulation()

#### Load in pybullet

In [ ]:
#load robot
robot_urdf = rl.datapath + '/urdf/talos_reduced.urdf';
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)

#load plane
plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,0), np.array([0,0,0,1]))

#set default visualization function
set_q_std = partial(set_q,robot_id, pb_joint_indices, set_base=True)
set_q_std(q0Complete)
vis_traj_std = partial(vis_traj, vis_func = set_q_std)

In [ ]:
_,_, ball_id = create_primitives(radius = 0.1)

#### Setup the environment

In [ ]:
task = 3 #choose 1, 2 or 3
brown_color = (165./255, 42/255., 42/255.,1.)
brown2_color = (165./255, 65/255., 65/255.,1.)
purple_color = (165./255, 100/255., 255/255.,.9)
ori = (0,0,0,1)

if task == 1:
    #task 1
    _,_, box_id = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.25, 0.4, 0.03], pos = [0.7,0,.95], rgbaColor=brown_color)
    col_ids = [plane_id, box_id]
elif task == 2:
    #task 2
    _,_, box_id = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.25, 0.4, 0.03], pos= [0.55,0,.8], rgbaColor=brown_color)
    _,_, box_id2 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.4, 0.12], pos = [0.33,0,.95],  rgbaColor=brown2_color)
    col_ids = [plane_id, box_id, box_id2]
elif task == 3:
        #task3
    _,_, box_id =  create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.25, 0.4, 0.03], pos = [0.7,0,.75], rgbaColor=brown_color)
    _,_, box_id2 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.4, 0.03], pos = [0.5,0,1.22], rgbaColor=purple_color)
    _,_, box_id3 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.25], pos = [0.5,0.3,1.], rgbaColor=purple_color)
    _,_, box_id4 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.25], pos = [0.5,-0.3,1.], rgbaColor=purple_color)
    _,_, box_id5 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.35], pos = [0.5,0.3,0.4], rgbaColor=brown_color)
    _,_, box_id6 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.35], pos = [0.5,-0.3,.4], rgbaColor=brown_color)
    _,_, box_id7 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.35], pos = [0.9,0.3,0.4], rgbaColor=brown_color)
    _,_, box_id8 = create_primitives(shapeType=p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.35], pos = [0.9,-0.3,.4], rgbaColor=brown_color)
    col_ids = [plane_id, box_id, box_id2, box_id3, box_id4, box_id5, box_id6]

#### Define frame indices

In [ ]:
pb_frame_names = [p.getJointInfo(robot_id,i)[1] for i in range(dof)]

In [ ]:
pb_lh_frame_id = pb_frame_names.index(b'gripper_left_joint')
pb_rh_frame_id = pb_frame_names.index(b'gripper_right_joint')
pb_lf_frame_id = pb_frame_names.index(b'leg_left_sole_fix_joint')
pb_rf_frame_id = pb_frame_names.index(b'leg_right_sole_fix_joint')

#### Load from pinocchio

In [ ]:
robot_urdf = rl.datapath + '/urdf/talos_reduced.urdf';
rmodel = pin.buildModelFromUrdf(robot_urdf , pin.JointModelFreeFlyer())
rdata = rmodel.createData()

robot_joint_limits = np.vstack([rmodel.lowerPositionLimit, rmodel.upperPositionLimit])

pin_frame_names = [f.name for f in rmodel.frames]
lh_frame_id = rmodel.getFrameId('gripper_left_joint')
rh_frame_id = rmodel.getFrameId('gripper_right_joint')
lf_frame_id = rmodel.getFrameId('leg_left_sole_fix_joint')
rf_frame_id = rmodel.getFrameId('leg_right_sole_fix_joint')

#### Define target poses

In [ ]:
#left foot
pos_lf, ori_lf = computePose(rmodel,rdata,lf_frame_id,q0Complete)
rpy_lf = mat2euler(ori_lf)
pose_lf_ref = np.concatenate([pos_lf, rpy_lf])
#right foot
pos_rf, ori_rf = computePose(rmodel,rdata,rf_frame_id,q0Complete)
rpy_rf = mat2euler(ori_rf)
pose_rf_ref = np.concatenate([pos_rf, rpy_rf])
#left hand
pos_lh, ori_lh = computePose(rmodel,rdata,lh_frame_id,q0Complete)
pose_lh_ref = np.concatenate([np.array([0.3,0.2, 0.4]), np.array([0,-np.pi/2,0.])])
#right hand
pos_rh, ori_rh = computePose(rmodel,rdata,rh_frame_id,q0Complete)
rpy_rh = mat2euler(ori_rh)
pose_rh_ref = np.concatenate([pos_rh, rpy_rh])

### Define Projector & IK solver

In [ ]:
lf_cost = CostFrameRPYFloatingBase(rmodel, rdata, pose_lf_ref,lf_frame_id, sel_vector=np.array([1,1,1,1,1,1]))
rf_cost = CostFrameRPYFloatingBase(rmodel, rdata, pose_rf_ref,rf_frame_id, sel_vector=np.array([1,1,1,1,1,1]))
bound_cost = CostBound(robot_joint_limits)
posture_cost = CostPosture(rmodel, rdata, q0Complete)
com_bounds = np.array([[-0.1,-0.1,0.6],[0.1, 0.1, 1.1]])
cost_com_bounds = CostCOMBounds(rmodel, rdata, com_bounds)
cost_sum = CostSum()

cost_sum.addCost(lf_cost, 20., 'lf_pose', 1e-5)
cost_sum.addCost(rf_cost, 20.,'rf_pose', 1e-5)
cost_sum.addCost(bound_cost, 10., 'com_limit', 1e-4)
cost_sum.addCost(cost_com_bounds, 10., 'joint_limit', 1e-4)
cost_sum.addCost(posture_cost, .01, 'posture', 1e3)

robot_projector = TalosCostProjector(cost_sum)


rh_cost = CostFrameRPYFloatingBase(rmodel, rdata, pose_rh_ref,rh_frame_id, sel_vector=np.array([1,1,1,0,0,0]))

cost_sum2 = CostSum()
cost_sum2.addCost(rh_cost, 20., 'rh_pose', 1e-3)
cost_sum2.addCost(lf_cost, 20., 'lf_pose', 1e-5)
cost_sum2.addCost(rf_cost, 20.,'rf_pose', 1e-5)
cost_sum2.addCost(bound_cost, 10., 'com_limit', 1e-4)
cost_sum2.addCost(cost_com_bounds, 10., 'joint_limit', 1e-4)
cost_sum2.addCost(posture_cost, .01, 'posture', 1e3)

robot_ik_solver = TalosCostProjector(cost_sum2)

#### Using GAN sampler

In [ ]:
class GANSampler():
    def __init__(self, target_sampler):
        self.target_sampler = target_sampler
    
    def sample(self,N = 1, _poses = None, var = 1., idx_input = None):
        if _poses is None:
            _poses = self.target_sampler.sample(N)
        self.poses = _poses
        _foot_poses = np.tile(np.concatenate([pose_rf_ref[:3], pose_lf_ref[:3]]), (N,1))
        _targets = np.hstack([_poses, _foot_poses])
        feed_dict={batch_size: N*N_net, batch_target: _targets, eps_var:var}
        feed_dict[batch_foot_ori] = _batch_foot_ori

        _samples_q = sess.run([samples_q],feed_dict)
        _samples_q = _samples_q[0]
        qnew = []
        for i in range(N):
            #samples from N_net networks
            if idx_input is None:
                idx = np.random.randint(N_net)
            else:
                idx = idx_input
            q = _samples_q[idx*N + i]
            qnew += [get_pb_config(q)]
        return np.array(qnew)

In [ ]:
#define standard sampler
base_sampler = sampler(com_bounds.copy())
joint_sampler = sampler(robot_joint_limits[:,7:])
rob_sampler = talos_sampler(base_sampler, np.array([0,0,0,1]), joint_sampler)

#define GAN sampler
target_bounds = np.array([[-0.6, -.6, 0.3], [0.8, .6, 1.4]])
#target_bounds = np.array([[0.2, -.6, 0.3], [0.8, .6, 1.4]])
target_sampler = sampler( target_bounds)
gan_sampler = GANSampler(target_sampler)

#### Try GAN sampler


In [ ]:
#q = gan_sampler.sample(_poses = np.array([[-0.2, -0.3, 1.]]))[0]
q = gan_sampler.sample()[0]
set_q_std(q)

#### Follow a given EE trajectory

In [ ]:
ps = []
for theta in np.linspace(0, np.pi*2, 36):
    p1 = 0.3*np.array([0, np.cos(theta), np.sin(theta)]) + np.array([0.5, -0.4, 1.])
    ps += [p1]
for theta in np.linspace(0, np.pi*2, 36):
    p1 = 0.3*np.array([0, 2-np.cos(theta), np.sin(theta)]) + np.array([0.5, -0.4, 1.])
    ps += [p1]
ps = np.array(ps)

In [ ]:
#Far targets
ps = []
radius = 0.5
for theta in np.linspace(0, np.pi*2, 36):
    p1 = radius*np.array([0, np.cos(theta), np.sin(theta)]) + np.array([0.5, -0.4, 1.8])
    ps += [p1]
for theta in np.linspace(0, np.pi*2, 36):
    p1 = radius*np.array([0, 2-np.cos(theta), np.sin(theta)]) + np.array([0.5, -0.4, 1.8])
    ps += [p1]
ps = np.array(ps)

In [ ]:
#Far targets(2) 
ps = []
radius = 0.8
for theta in np.linspace(0, np.pi*2, 36):
    p1 = radius*np.array([0, .7*np.cos(theta), np.sin(2*theta)]) + np.array([0.5, -0.4, 1.6])
    ps += [p1]
for theta in np.linspace(0, np.pi*2, 36):
    p1 = radius*np.array([0, .7*(2-np.cos(theta)), np.sin(2*theta)]) + np.array([0.5, -0.4, 1.6])
    ps += [p1]
ps = np.array(ps)

In [ ]:
idx_input = 0
qs = gan_sampler.sample(_poses = ps, N=len(ps), var = .0001, idx_input=idx_input)
for i,q in enumerate(qs):
    set_q_std(q)
    p.resetBasePositionAndOrientation(ball_id, ps[i], (0,0,0,1))
    time.sleep(0.2)

In [ ]:
make_animation(qs, ps, 'data/video_follow_traj_far' + str(idx_input) + '.mp4')

#### Set Camera POV

In [ ]:
p.resetDebugVisualizerCamera( cameraDistance=2.5, cameraYaw=90, cameraPitch=-20, cameraTargetPosition=[0,0,1])

In [ ]:
p.resetDebugVisualizerCamera( cameraDistance=1.8, cameraYaw=0, cameraPitch=-20, cameraTargetPosition=[0,0,1])

#### Try random

In [ ]:
N = 10
target_samples = target_sampler.sample(N = N)
samples = gan_sampler.sample(_poses = target_samples, N=len(target_samples), var=0.1)
idx = 0
for i in range(N):
    p.resetBasePositionAndOrientation(ball_id, target_samples[i], (0,0,0,1))
    for j in range(2):
        save_screenshot(200, 200, 800, 1000, 'data/temp'+str(idx)+'.png', to_show=False)
        idx +=1
    set_q_std(samples[i])
    for j in range(4):
        save_screenshot(200, 200, 800, 1000, 'data/temp'+str(idx)+'.png', to_show=False)
        idx +=1

os.system('ffmpeg -r 4 -start_number 0 -i data/temp%d.png -c:v libx264 -r 30 -pix_fmt yuv420p ' + 'data/video_random_talos.mp4')
os.system('rm data/temp*')

In [ ]:
def make_animation(qs,targets, filename='data/videos.mp4'):
    for i,q in enumerate(qs):
        p.resetBasePositionAndOrientation(ball_id, targets[i], (0,0,0,1))
        set_q_std(q)
        save_screenshot(200, 200, 800, 1000, 'data/temp'+str(i)+'.png', to_show=False)
    #make into videos
    os.system('ffmpeg -r 10 -start_number 0 -i data/temp%d.png -c:v libx264 -r 30 -pix_fmt yuv420p ' + filename)
    os.system('rm data/temp*')

#### Compare the projector

In [ ]:
methods = [rob_sampler, gan_sampler]
method_names = ['Random', 'GAN']

samples = gan_sampler.sample(1000)

N = 10

data = dict()
for m in range(len(methods)):
    comp_times = []
    success = []
    fevals = []
    for i in range(N):
        #idx = np.random.randint(500)
        idx = i
        if method_names[m] == 'GAN':
            q = samples[idx]
        else:
            q = rob_sampler.sample().flatten()
        tic = time.time()
        res = robot_projector.project(q)
        toc = time.time()
        comp_times += [toc-tic]
        success += [res['stat']]
        fevals += [res['nfev']]
    data[method_names[m]] = [comp_times, success, fevals]

In [ ]:
for m in range(len(methods[:2])):
    print('& ' + method_names[m], end=' ')
    comp_times, success, fevals = data[method_names[m]]
    comp_times = np.array(comp_times)
    fevals = np.array(fevals)

    print('& {0:.1f} &  {1:.1f} $\pm$ {2:.1f} &  {3:.1f} $\pm$ {4:.1f} &  {5:.1f} $\pm$ {6:.1f}'.format(np.sum(success)*100./N, np.mean(comp_times)*1000, np.std(comp_times)*1000, np.mean(comp_times[success])*1000, np.std(comp_times[success])*1000,np.mean(fevals[success]), np.std(fevals[success])), end = ''),
    print('\\\\ ')

### Compare the IK solver

In [ ]:
targets = data_target[:2000,:3]
samples = gan_sampler.sample(N=2000, _poses=targets, var = 0.1)

In [ ]:
N = 10
data = dict()
for m in range(len(methods)):
    comp_times = []
    success = []
    fevals = []
    for i in range(N):
        idx = i
        if method_names[m] == 'GAN':
            q = samples[idx]
        else:
            q = rob_sampler.sample().flatten()
        tic = time.time()
        robot_ik_solver.cost.costs['rh_pose'].cost.desired_pose[:3] = targets[i]
        res = robot_ik_solver.project(q)
        toc = time.time()
        comp_times += [toc-tic]
        success += [res['stat']]
        fevals += [res['nfev']]
        data[method_names[m]] = [comp_times, success, fevals]
clear_output()

In [ ]:
for m in range(len(methods[:2])):
    print('& ' + method_names[m], end=' ')
    comp_times, success, fevals = data[method_names[m]]
    comp_times = np.array(comp_times)
    fevals = np.array(fevals)

    print('& {0:.1f} &  {1:.1f} $\pm$ {2:.1f} &  {3:.1f} $\pm$ {4:.1f} &  {5:.1f} $\pm$ {6:.1f}'.format(np.sum(success)*100./N, np.mean(comp_times)*1000, np.std(comp_times)*1000, np.mean(comp_times[success])*1000, np.std(comp_times[success])*1000,np.mean(fevals[success]), np.std(fevals[success])), end = ''),
    print('\\\\ ')

***

## c-RRT with Talos

In [ ]:
rob_col_checker = col_checker(robot_id, pb_joint_indices, col_ids, omit_indices=[ 50, 57], floating_base = True) #omit the collision of the plane with the feet
rob_interpolator = interpolator()

In [ ]:
rrt = cRRT(rmodel.nq, rob_sampler, rob_col_checker, rob_interpolator, robot_projector)

ganrrt = cRRT(rmodel.nq, gan_sampler, rob_col_checker, rob_interpolator, robot_projector)

hybrid_sampler = HybridSampler(rob_sampler, gan_sampler, p_random=0.2)
hybridrrt = cRRT(rmodel.nq, hybrid_sampler, rob_col_checker, rob_interpolator, robot_projector)

#### Get initial state

In [ ]:
if task == 1:
    #Task 1
    init_bounds = np.array([[0.55, -0.25, 1.1], [0.85, 0.25, 1.3]])
    goal_bounds = np.array([[0.55, -0.25, .4], [0.85, 0.25, .8]])
elif task == 2:
    init_bounds = np.array([[0.3, -0.25, 1.], [0.7, 0.25, 1.3]])
    goal_bounds = np.array([[0.3, -0.25, .4], [0.7, 0.25, .8]])
elif task == 3:
    init_bounds = np.array([[0.55, -0.25, .8], [0.7, 0.25, 1.1]])
    goal_bounds = np.array([[0.55, -0.25, .3], [0.7, 0.25, .6]])
init_sampler = sampler(init_bounds)
goal_sampler = sampler(goal_bounds)

#### Try sampling the tasks

#### Sample initial and goal configurations

In [ ]:
def get_valid_config(pos_sampler, goal_pose = None, max_try = 50):
    is_collide = True
    status = False
    i = 0
    while is_collide or status is False and i < max_try:
        i+=1
        if goal_pose is None: 
            goal_pos = pos_sampler.sample()
        else:
            goal_pos = goal_pose.copy()
        #goal_pos = np.array([0.6, -0.1, .9])[None,:]
        #sample = gan_sampler.sample(_poses=goal_pos, var=1.)[0]
        sample = rob_sampler.sample()[0]
        set_q_std(sample)
        rh_cost.desired_pose[:3] = goal_pos[0]
        res = robot_ik_solver.project(sample.flatten())
        start_state = res['q']
        status = res['stat']
        is_collide = rob_col_checker.check_collision(start_state.flatten())
            
            
    set_q_std(start_state)
    clear_output()
    return start_state, goal_pos

def get_multiple_configs(pos_sampler, N = 1, R = 1):
    samples_set = []
    for i in range(N):
        print(i)
        sample, goal_pos = get_valid_config(pos_sampler)
        samples = [sample]
        for j in range(R-1):
            sample, goal_pos = get_valid_config(pos_sampler, goal_pos)
            samples += [sample]
        samples_set += [samples]
    clear_output()
    return samples_set

In [ ]:
is_generate_data = True
N = 5 #number of initial configurations
K = 5 #number of goal configurations per task

if is_generate_data:
    init_configs = get_multiple_configs(init_sampler, N = N)
    goal_configs = get_multiple_configs(goal_sampler, N = N, R = K)

    data_configs = dict()
    data_configs['init'] = init_configs
    data_configs['goal'] = goal_configs
    np.save('data/task1' + str(task) + '_configs.npy', data_configs)
else:
    data_configs = np.load('data/task1' + str(task) + '_configs.npy', allow_pickle=True).tolist()
    init_configs = data_configs['init']
    goal_configs = data_configs['goal']

'''
for qs in init_configs:
    for q in qs:
        set_q_std(q)
        input()
clear_output()

for qs in goal_configs:
    for q in qs:
        set_q_std(q)
        input()
clear_output()
'''


## Planning comparison

In [ ]:
gan_result = dict()
std_result = dict()
hybrid_result = dict()
results = [gan_result, std_result, hybrid_result]
for result in results:
    result['comp_times'] = []
    result['nfevs'] = []
    result['retry'] = []
    result['nexts'] = []
    result['success'] = []
    result['path'] = []

i=0

In [ ]:
for qs in init_configs[:]:
    start_state = qs[0]
    for goal_states in goal_configs[:]:
        i+=1
        print(i, np.mean(std_result['comp_times']), np.mean(gan_result['comp_times']), np.mean(hybrid_result['comp_times']))
        print(i, np.sum(std_result['success']), np.sum(gan_result['success']), np.sum(hybrid_result['success']))
        
        
        #plan using crrt
        path, nfevs, nexts, success, retry, t  = rrt.plan(start_state,goal_states, max_extension_steps=500)
        clear_output()
        std_result['comp_times'] += [t]
        std_result['nfevs'] += [nfevs]
        std_result['retry'] += [retry]
        std_result['nexts'] += [nexts]
        std_result['success'] += [success]
        std_result['path'] += [path]
        
        
        #plan using ganrrt
        path, nfevs, nexts, success, retry, t  = ganrrt.plan(start_state,goal_states, max_extension_steps=500)
        clear_output()
        gan_result['comp_times'] += [t]
        gan_result['nfevs'] += [nfevs]
        gan_result['retry'] += [retry]
        gan_result['nexts'] += [nexts]
        gan_result['success'] += [success]
        gan_result['path'] += [path]
        
        #plan using hybridrrt
        path, nfevs, nexts, success, retry, t  = hybridrrt.plan(start_state,goal_states, max_extension_steps=500)
        clear_output()
        hybrid_result['comp_times'] += [t]
        hybrid_result['nfevs'] += [nfevs]
        hybrid_result['retry'] += [retry]
        hybrid_result['nexts'] += [nexts]
        hybrid_result['success'] += [success]
        hybrid_result['path'] += [path]
        
        if i % 20 == 0:
            data = dict()
            data['random'] = std_result
            data['gan'] = gan_result
            data['hybrid'] = hybrid_result
            np.save('data/task3_temp.npy', data)

In [ ]:
print('& Random & {0:.1f} & {1:.2f} $\pm$ {2:.2f} & {3:.1f} $\pm$ {4:.1f} & {5:.1f} $\pm$ {6:.1f} \\\\'.format(np.sum(std_result['success'])*100./len(std_result['success']), np.mean(std_result['comp_times']), np.std(std_result['comp_times']), np.mean(std_result['nfevs']), np.std(std_result['nfevs']), np.mean(std_result['nexts']), np.std(std_result['nexts'])))

print('& GAN & {0:.1f} & {1:.2f} $\pm$ {2:.2f} & {3:.1f} $\pm$ {4:.1f} & {5:.1f} $\pm$ {6:.1f} \\\\'.format(np.sum(gan_result['success'])*100./len(gan_result['success']), np.mean(gan_result['comp_times']), np.std(gan_result['comp_times']), np.mean(gan_result['nfevs']), np.std(gan_result['nfevs']), np.mean(gan_result['nexts']), np.std(gan_result['nexts'])))

print('& Hybrid & {0:.1f} & {1:.2f} $\pm$ {2:.2f} & {3:.1f} $\pm$ {4:.1f} & {5:.1f} $\pm$ {6:.1f} \\\\'.format(np.sum(hybrid_result['success'])*100./len(hybrid_result['success']), np.mean(hybrid_result['comp_times']), np.std(hybrid_result['comp_times']), np.mean(hybrid_result['nfevs']), np.std(hybrid_result['nfevs']), np.mean(hybrid_result['nexts']), np.std(hybrid_result['nexts'])))

In [ ]:
print('& Random & {0:.1f} & {1:.2f} $\pm$ {2:.2f} & {3:.1f} $\pm$ {4:.1f} & {5:.1f} $\pm$ {6:.1f} \\\\'.format(np.sum(std_result['success'])*100./len(std_result['success']), np.mean(std_result['comp_times']), np.std(std_result['comp_times']), np.mean(std_result['nfevs']), np.std(std_result['nfevs']), np.mean(std_result['nexts']), np.std(std_result['nexts'])))

print('& GAN & {0:.1f} & {1:.2f} $\pm$ {2:.2f} & {3:.1f} $\pm$ {4:.1f} & {5:.1f} $\pm$ {6:.1f} \\\\'.format(np.sum(gan_result['success'])*100./len(gan_result['success']), np.mean(gan_result['comp_times']), np.std(gan_result['comp_times']), np.mean(gan_result['nfevs']), np.std(gan_result['nfevs']), np.mean(gan_result['nexts']), np.std(gan_result['nexts'])))

print('& Hybrid & {0:.1f} & {1:.2f} $\pm$ {2:.2f} & {3:.1f} $\pm$ {4:.1f} & {5:.1f} $\pm$ {6:.1f} \\\\'.format(np.sum(hybrid_result['success'])*100./len(hybrid_result['success']), np.mean(hybrid_result['comp_times']), np.std(hybrid_result['comp_times']), np.mean(hybrid_result['nfevs']), np.std(hybrid_result['nfevs']), np.mean(hybrid_result['nexts']), np.std(hybrid_result['nexts'])))

In [ ]:
data = dict()
data['random'] = std_result
data['gan'] = gan_result
data['hybrid'] = hybrid_result
np.save('data/task' + str(task) + '3.npy', data)